# Previous code

In [4]:
import torch
import torch.nn.functional as F

In [5]:
torch.manual_seed(42)

In [6]:
words = open("names.txt", "r").read().splitlines()
words[:10]

['emma',
 'olivia',
 'ava',
 'isabella',
 'sophia',
 'charlotte',
 'mia',
 'amelia',
 'harper',
 'evelyn']

In [7]:
# build vocabulary

chars = ["."] + sorted(list(set("".join(words))))
stoi = { s:i for i,s in enumerate(chars) } # string to integer
itos = { i:s for s,i in stoi.items() } # integer to string

In [11]:
context_length = 3

In [12]:
def build_dataset(words):
  X, Y = [], []

  for w in words:
    context = [0] * context_length
    for ch in w + ".":
      ix = stoi[ch]
      X.append(context)
      Y.append(ix)
      context = context[1:] + [ix]

  X = torch.tensor(X)
  Y = torch.tensor(Y)
  print(X.shape, Y.shape)
  return X, Y

In [13]:
# build training, dev, test sets

import random

random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr, Ytr = build_dataset(words[:n1])
Xdev, Ydev = build_dataset(words[n1:n2])
Xte, Yte = build_dataset(words[n2:])

torch.Size([182625, 3]) torch.Size([182625])
torch.Size([22655, 3]) torch.Size([22655])
torch.Size([22866, 3]) torch.Size([22866])


# Initial code

In [24]:
class Linear:

  def __init__(self, fan_in, fan_out, bias=True):
    self.weight = torch.randn((fan_in, fan_out)) / fan_in**0.5 # kaiming init
    self.bias = torch.zeros(fan_out) if bias else None

  def __call__(self, x):
    self.out = x @ self.weight
    if self.bias is not None:
      self.out += self.bias
    return self.out

  def parameters(self):
    return [self.weight] + ([] if self.bias is None else [self.bias])

In [25]:
class BatchNorm1d:

  def __init__(self, dim, eps=1e-5, momentum=0.1):
    self.eps = eps # to avoid divisionbyzero error when scaling
    self.momentum = momentum # to compute running average
    self.training = True
    # parameters (trained with backprop)
    self.gamma = torch.ones(dim)
    self.beta = torch.zeros(dim)
    # buffers (stored with a running momentum update)
    self.running_mean = torch.zeros(dim)
    self.running_var = torch.ones(dim)

  def __call__(self, x):
    # calculate forward pass
    if self.training:
      xmean = x.mean(0, keepdim=True) # batch mean
      xvar = x.var(0, keepdim=True) # batch variance
    else:
      xmean = self.running_mean
      xvar = self.running_var
    xhat = (x-xmean) / torch.sqrt(xvar + self.eps)
    self.out = self.gamma * xhat + self.beta
    # update the bufers
    if self.training:
      with torch.no_grad():
        self.running_mean = (1-self.momentum) * self.running_mean + self.momentum * xmean
        self.running_var = (1-self.momentum) * self.running_var + self.momentum * xvar
    return self.out

  def parameters(self):
    return [self.gamma, self.beta]

In [26]:
class Tanh:

  def __call__(self, x):
    self.out = torch.tanh(x)
    return self.out

  def parameters(self):
    return []

In [34]:
class Embedding:
    def __init__(self, vocab_size, embed_dim):
        self.weight = torch.randn((vocab_size, embed_dim))

    def __call__(self, IX):
        self.out = self.weight[IX]
        return self.out
    
    def parameters(self):
        return [self.weight]
    
class Flatten:
    def __call__(self, x):
        self.out = x.view(x.shape[0], -1)
        return self.out
    
    def parameters(self):
        return []

In [39]:
class Sequential:
    def __init__(self, layers):
        self.layers = layers

    def __call__(self, x):
        for layer in self.layers:
            x = layer(x)
        self.out = x
        return self.out
    
    def parameters(self):
        return [p for layer in self.layers for p in layer.parameters()]

In [49]:
n_embed = 10
n_hidden = 100
vocab_size = 27

model = Sequential([
  Embedding(vocab_size, n_embed),
  Flatten(),
  Linear(n_embed * context_length, n_hidden), Tanh(),
  Linear(                n_hidden, n_hidden), Tanh(),
  Linear(                n_hidden, n_hidden), Tanh(),
  Linear(                n_hidden, n_hidden), Tanh(),
  Linear(                n_hidden, n_hidden), Tanh(),
  Linear(                n_hidden, vocab_size)
])

with torch.no_grad():
  # last layer: make less confident
  model.layers[-1].weight *= 0.1
  # # all other layers: apply gain
  # for layer in model.layers[:-1]:
  #   if isinstance(layer, Linear):
  #     layer.weight *= 5/3 # only because we are using tanh

parameters = model.parameters()
print(sum(p.nelement() for p in parameters))
for p in parameters:
  p.requires_grad = True

46497


In [50]:
max_steps = 30_000
batch_size = 128

for i in range(1, max_steps+1):
  # construct minibatch
  ix = torch.randint(0, Xtr.shape[0], (batch_size,))
  Xb, Yb = Xtr[ix], Ytr[ix]

  # forward pass
  logits = model(Xb)
  loss = F.cross_entropy(logits, Yb)

  # backward pass
  for p in parameters:
    p.grad = None
  loss.backward()

  # update
  lr = 0.1 if i < 100_000 else 0.01
  for p in parameters:
    p.data -= lr * p.grad

  # track stats
  if i % 10_000 == 0:
    print(f"{i:7d}/{max_steps:7d}: {loss.item():.4f}")

      0/  30000: 3.3031
  10000/  30000: 2.1060
  20000/  30000: 2.0774


In [51]:
for layer in model.layers:
    layer.training = False

In [52]:
@torch.no_grad()
def split_loss(split):
    x, y = {
        "train": (Xtr, Ytr),
        "val": (Xdev, Ydev),
        "test": (Xte, Yte)
    }[split]
    logits = model(x)
    loss = F.cross_entropy(logits, y)
    print(split, loss.item())

In [53]:
split_loss("train")
split_loss("val")

train 2.0458154678344727
val 2.120760917663574


In [54]:
# sample from the model

for _ in range(20):
    out = []
    context = [0] * context_length
    while True:
        logits = model(torch.tensor([context]))
        probs = F.softmax(logits, dim=1)
        # sample from distribution
        ix = torch.multinomial(probs, num_samples=1).item()
        # shift the context window and track the samples
        context = context[1:] + [ix]
        out.append(ix)
        # if we sample the "." token, break
        if ix == 0:
            break

    print("".join(itos[i] for i in out))

zaxie.
alviena.
yenna.
mah.
sammitoco.
kiya.
blaterri.
kyma.
edvraxtyn.
ekatha.
zonaf.
davimmilan.
brayn.
elyston.
ten.
fylie.
cyossoyaan.
demba.
zei.
sorvectella.
